In [0]:
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql import SparkSession, functions as S
spark = SparkSession.builder.getOrCreate()



In [0]:
# Importando o arquivo
df=spark.read.format("csv")\
.option("header","true")\
.option("sep",";")\
.option("inferSchema","True")\
.load("dbfs:/FileStore/shared_uploads/leonildo.silva@tmf-group.com/base_mensalizada_de_funcionarios-1.csv")

In [0]:
# Visualizando o schema das colunas
df.printSchema()

root
 |-- mes_referencia: string (nullable = true)
 |-- id_funcionario: integer (nullable = true)
 |-- data_de_nascimento: string (nullable = true)
 |-- data_de_admissao: string (nullable = true)
 |-- data_de_demissao: string (nullable = true)
 |-- grau_de_instrucao: string (nullable = true)
 |-- cargo: string (nullable = true)
 |-- salario: string (nullable = true)



In [0]:
# Visualizando os dados 
df.show()

+--------------+--------------+------------------+----------------+----------------+--------------------+--------------------+-------+
|mes_referencia|id_funcionario|data_de_nascimento|data_de_admissao|data_de_demissao|   grau_de_instrucao|               cargo|salario|
+--------------+--------------+------------------+----------------+----------------+--------------------+--------------------+-------+
|    01/01/2020|         11223|        01/04/1995|      01/06/2018|            null|   4ª série completo|operador de colhe...|   1500|
|    01/01/2020|         13456|        05/03/1960|      11/02/2002|            null|   ensino médio comp|    assistente de rh|   1400|
|    01/01/2020|         12345|        02/11/1980|            null|            null|ensino fundamenta...|operador de colhe...|   1800|
|    01/01/2020|         24457|        26/07/2000|      10/05/2020|            null|            mestrado| gerente de produção|  11500|
|    01/01/2020|         13254|        22/05/1977|     

In [0]:
# Criando uma view para analisar os dados usando sql 
df.createOrReplaceTempView("funcionarios")

In [0]:
%sql
--Analisando a quantidade duplicidades

select mes_referencia,id_funcionario,count(*) from funcionarios
group by mes_referencia,id_funcionario
having count(*)>1

mes_referencia,id_funcionario,count(1)
01/07/2020,12345,2
01/04/2020,13456,2
01/06/2020,46457,2
01/01/2020,46578,2
01/05/2020,13254,2
01/07/2020,57687,2


In [0]:
# Analisando a quantidade de dados nulos de cada coluna
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+--------------+--------------+------------------+----------------+----------------+-----------------+-----+-------+
|mes_referencia|id_funcionario|data_de_nascimento|data_de_admissao|data_de_demissao|grau_de_instrucao|cargo|salario|
+--------------+--------------+------------------+----------------+----------------+-----------------+-----+-------+
|             0|             0|                 3|               4|             119|                0|    0|      0|
+--------------+--------------+------------------+----------------+----------------+-----------------+-----+-------+



In [0]:
# Filtrando quais valores estão nulos na coluna data_de_nascimento
df.filter("data_de_nascimento is null").show()

+--------------+--------------+------------------+----------------+----------------+--------------------+--------------------+-------+
|mes_referencia|id_funcionario|data_de_nascimento|data_de_admissao|data_de_demissao|   grau_de_instrucao|               cargo|salario|
+--------------+--------------+------------------+----------------+----------------+--------------------+--------------------+-------+
|    01/02/2020|         13254|              null|      03/09/2005|            null|ensino medio inco...|operador de colhe...|   1625|
|    01/05/2020|         46578|              null|      26/11/2009|            null|ensino superior c...|analista de siste...|   7500|
|    01/07/2020|         12345|              null|            null|            null|ensino fundamenta...|operador de colhe...|   1950|
+--------------+--------------+------------------+----------------+----------------+--------------------+--------------------+-------+



In [0]:
# Substituir os valores nulos por dados fixos (colunas como data de admissão, data de nascimento, id do funcionário, grau de instrução, cargo e salário não podem conter valores nulos)
df=df.na.fill(value='01/01/1800',subset=["mes_referencia"])
df=df.na.fill(value=0,subset=["Id_funcionario"])
df=df.na.fill(value='01/01/1800',subset=["data_de_nascimento"])
df=df.na.fill(value='01/01/1800',subset=["data_de_admissao"])
df=df.na.fill(value="Nao cadastrado",subset=["grau_de_instrucao"])
df=df.na.fill(value="Nao cadastrado",subset=["cargo"])
df=df.na.fill(value=0,subset=["salario"])
df.show()

+--------------+--------------+------------------+----------------+----------------+--------------------+--------------------+-------+
|mes_referencia|id_funcionario|data_de_nascimento|data_de_admissao|data_de_demissao|   grau_de_instrucao|               cargo|salario|
+--------------+--------------+------------------+----------------+----------------+--------------------+--------------------+-------+
|    01/01/2020|         11223|        01/04/1995|      01/06/2018|            null|   4ª série completo|operador de colhe...|   1500|
|    01/01/2020|         13456|        05/03/1960|      11/02/2002|            null|   ensino médio comp|    assistente de rh|   1400|
|    01/01/2020|         12345|        02/11/1980|      01/01/1800|            null|ensino fundamenta...|operador de colhe...|   1800|
|    01/01/2020|         24457|        26/07/2000|      10/05/2020|            null|            mestrado| gerente de produção|  11500|
|    01/01/2020|         13254|        22/05/1977|     

In [0]:
# Analisando a quantidade de dados nulos de cada coluna
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+--------------+--------------+------------------+----------------+----------------+-----------------+-----+-------+
|mes_referencia|id_funcionario|data_de_nascimento|data_de_admissao|data_de_demissao|grau_de_instrucao|cargo|salario|
+--------------+--------------+------------------+----------------+----------------+-----------------+-----+-------+
|             0|             0|                 0|               0|             119|                0|    0|      0|
+--------------+--------------+------------------+----------------+----------------+-----------------+-----+-------+



In [0]:
# Formatando as colunas
df = df.withColumn("mes_referencia",F.to_date(col("mes_referencia"),"dd/MM/yyyy")) \
.withColumn("data_de_nascimento",F.to_date(col("data_de_nascimento"),"dd/MM/yyyy")) \
.withColumn("data_de_admissao",F.to_date(col("data_de_admissao"),"dd/MM/yyyy")) \
.withColumn("data_de_demissao",F.to_date(col("data_de_demissao"),"dd/MM/yyyy"))\
.withColumn("Id_funcionario",df["Id_funcionario"].cast('integer'))\
.withColumn("salario",df["salario"].cast('float'))

df.show()

+--------------+--------------+------------------+----------------+----------------+--------------------+--------------------+-------+
|mes_referencia|Id_funcionario|data_de_nascimento|data_de_admissao|data_de_demissao|   grau_de_instrucao|               cargo|salario|
+--------------+--------------+------------------+----------------+----------------+--------------------+--------------------+-------+
|    2020-01-01|         11223|        1995-04-01|      2018-06-01|            null|   4ª série completo|operador de colhe...| 1500.0|
|    2020-01-01|         13456|        1960-03-05|      2002-02-11|            null|   ensino médio comp|    assistente de rh| 1400.0|
|    2020-01-01|         12345|        1980-11-02|      1800-01-01|            null|ensino fundamenta...|operador de colhe...| 1800.0|
|    2020-01-01|         24457|        2000-07-26|      2020-05-10|            null|            mestrado| gerente de produção|11500.0|
|    2020-01-01|         13254|        1977-05-22|     

In [0]:
# Analisando a quantidade de dados nulos de cada coluna novamente após a formatação

df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()



+--------------+--------------+------------------+----------------+----------------+-----------------+-----+-------+
|mes_referencia|Id_funcionario|data_de_nascimento|data_de_admissao|data_de_demissao|grau_de_instrucao|cargo|salario|
+--------------+--------------+------------------+----------------+----------------+-----------------+-----+-------+
|             0|             0|                 0|               0|             119|                0|    0|      1|
+--------------+--------------+------------------+----------------+----------------+-----------------+-----+-------+



In [0]:
#Verificando qual linha ficou nula na coluna salario após a formatação.
df.filter("salario is null").show()

+--------------+--------------+------------------+----------------+----------------+-----------------+-------------------+-------+
|mes_referencia|Id_funcionario|data_de_nascimento|data_de_admissao|data_de_demissao|grau_de_instrucao|              cargo|salario|
+--------------+--------------+------------------+----------------+----------------+-----------------+-------------------+-------+
|    2020-11-01|         95847|        1993-04-23|      2012-04-25|            null|        graduação|líder de tecnologia|   null|
+--------------+--------------+------------------+----------------+----------------+-----------------+-------------------+-------+



In [0]:
# Substituir o valor nulo da coluna salario.
df=df.na.fill(value=0,subset=["salario"])


In [0]:
#Verificando se a coluna salario possui valor nulo
df.filter("salario is null").show()

+--------------+--------------+------------------+----------------+----------------+-----------------+-----+-------+
|mes_referencia|Id_funcionario|data_de_nascimento|data_de_admissao|data_de_demissao|grau_de_instrucao|cargo|salario|
+--------------+--------------+------------------+----------------+----------------+-----------------+-----+-------+
+--------------+--------------+------------------+----------------+----------------+-----------------+-----+-------+



In [0]:
# Retirando as duplicidades
df = df.distinct()
display(df)

mes_referencia,Id_funcionario,data_de_nascimento,data_de_admissao,data_de_demissao,grau_de_instrucao,cargo,salario
2020-09-01,46457,1982-02-12,1998-11-07,null,pós graduação,analista de BI,5900.0
2020-02-01,13456,1960-03-05,2002-02-11,null,ensino médio comp,assistente de rh,1400.0
2020-07-01,24457,2000-07-26,2020-05-10,null,mestrado,gerente de produção,12100.0
2020-08-01,59393,1998-05-17,2019-01-08,null,ensino médio completo,assistente administrativo,1520.0
2020-12-01,46578,1972-01-11,2009-11-26,null,ensino superior completo,analista de sistemas senior,8300.0
2020-06-01,24457,2000-07-26,2020-05-10,null,mestrado,gerente de produção,12100.0
2020-04-01,11223,1995-04-01,2018-06-01,null,4ª série completo,operador de colheita I,1500.0
2020-05-01,24457,2000-07-26,2020-05-10,null,mestrado,gerente de produção,11500.0
2020-05-01,57687,1991-05-28,2015-10-18,null,doutorado,cientista de dados,7200.0
2020-09-01,13456,1960-03-05,2002-02-11,null,ensino médio comp,assistente de rh,1520.0


In [0]:
# Valores do tipo texto com a primeira letra maiscula 
df =  df.withColumn("cargo", initcap(col('cargo'))) \
.withColumn("grau_de_instrucao", initcap(col('grau_de_instrucao')))

In [0]:
# Remover espaços das colunas string (grau_de_instrucao e cargo)
df = df.withColumn("grau_de_instrucao", trim(df.grau_de_instrucao))\
.withColumn("cargo", trim(df.cargo))\
.withColumn("cargo", expr("BTRIM(cargo, ' \t')"))\
.withColumn("grau_de_instrucao", expr("BTRIM(grau_de_instrucao, ' \t')"))



In [0]:
display(df)

mes_referencia,Id_funcionario,data_de_nascimento,data_de_admissao,data_de_demissao,grau_de_instrucao,cargo,salario
2020-09-01,46457,1982-02-12,1998-11-07,null,Pós Graduação,Analista De Bi,5900.0
2020-02-01,13456,1960-03-05,2002-02-11,null,Ensino Médio Comp,Assistente De Rh,1400.0
2020-07-01,24457,2000-07-26,2020-05-10,null,Mestrado,Gerente De Produção,12100.0
2020-08-01,59393,1998-05-17,2019-01-08,null,Ensino Médio Completo,Assistente Administrativo,1520.0
2020-12-01,46578,1972-01-11,2009-11-26,null,Ensino Superior Completo,Analista De Sistemas Senior,8300.0
2020-06-01,24457,2000-07-26,2020-05-10,null,Mestrado,Gerente De Produção,12100.0
2020-04-01,11223,1995-04-01,2018-06-01,null,4ª Série Completo,Operador De Colheita I,1500.0
2020-05-01,24457,2000-07-26,2020-05-10,null,Mestrado,Gerente De Produção,11500.0
2020-05-01,57687,1991-05-28,2015-10-18,null,Doutorado,Cientista De Dados,7200.0
2020-09-01,13456,1960-03-05,2002-02-11,null,Ensino Médio Comp,Assistente De Rh,1520.0


In [0]:
# criando uma view para analisar os dados usando sql no passo 3.
df.createOrReplaceTempView("funcionarios")


In [0]:
%sql
----- [TESTE EM SQL] Incluindo a regra para criação da coluna admitidos no mês
select mes_referencia,data_de_admissao, 
CASE
WHEN (YEAR(mes_referencia) = YEAR(data_de_admissao) and MONTH(mes_referencia) = MONTH(data_de_admissao))
then 1 else 0 End as admitido_no_mes
from funcionarios 

mes_referencia,data_de_admissao,admitido_no_mes
2020-09-01,1998-11-07,0
2020-02-01,2002-02-11,0
2020-07-01,2020-05-10,0
2020-08-01,2019-01-08,0
2020-12-01,2009-11-26,0
2020-06-01,2020-05-10,0
2020-04-01,2018-06-01,0
2020-05-01,2020-05-10,1
2020-05-01,2015-10-18,0
2020-09-01,2002-02-11,0


In [0]:
# [APLICAÇÃO DE REGRA] Incluindo a regra para criação da coluna admitidos no mês
df=df.withColumn("admitido_no_mes", expr("CASE WHEN (YEAR(mes_referencia) = YEAR(data_de_admissao) and MONTH(mes_referencia) = MONTH(data_de_admissao))  THEN 1 ELSE 0 END"))
df.show(116)



+--------------+--------------+------------------+----------------+----------------+--------------------+--------------------+-------+---------------+
|mes_referencia|Id_funcionario|data_de_nascimento|data_de_admissao|data_de_demissao|   grau_de_instrucao|               cargo|salario|admitido_no_mes|
+--------------+--------------+------------------+----------------+----------------+--------------------+--------------------+-------+---------------+
|    2020-09-01|         46457|        1982-02-12|      1998-11-07|            null|       Pós Graduação|      Analista De Bi| 5900.0|              0|
|    2020-02-01|         13456|        1960-03-05|      2002-02-11|            null|   Ensino Médio Comp|    Assistente De Rh| 1400.0|              0|
|    2020-07-01|         24457|        2000-07-26|      2020-05-10|            null|            Mestrado| Gerente De Produção|12100.0|              0|
|    2020-08-01|         59393|        1998-05-17|      2019-01-08|            null|Ensino Méd

In [0]:
%sql
-----[TESTE EM SQL] Incluindo a regra para criação da coluna demitidos no mês

select mes_referencia,data_de_demissao, 
CASE
WHEN (YEAR(mes_referencia) = YEAR(data_de_demissao) and MONTH(mes_referencia) = MONTH(data_de_demissao))
then 1 else 0 End as Demitido_no_mes
from funcionarios 

mes_referencia,data_de_demissao,Demitido_no_mes
2020-09-01,null,0
2020-02-01,null,0
2020-07-01,null,0
2020-08-01,null,0
2020-12-01,null,0
2020-06-01,null,0
2020-04-01,null,0
2020-05-01,null,0
2020-05-01,null,0
2020-09-01,null,0


In [0]:
# [APLICAÇÃO DE REGRA] Incluindo a regra para criação da coluna demitidos no mês
df=df.withColumn("demitido_no_mes", expr("CASE WHEN (YEAR(mes_referencia) = YEAR(data_de_demissao) and MONTH(mes_referencia) = MONTH(data_de_demissao))  THEN 1 ELSE 0 END"))
df.show(116)

+--------------+--------------+------------------+----------------+----------------+--------------------+--------------------+-------+---------------+---------------+
|mes_referencia|Id_funcionario|data_de_nascimento|data_de_admissao|data_de_demissao|   grau_de_instrucao|               cargo|salario|admitido_no_mes|demitido_no_mes|
+--------------+--------------+------------------+----------------+----------------+--------------------+--------------------+-------+---------------+---------------+
|    2020-09-01|         46457|        1982-02-12|      1998-11-07|            null|       Pós Graduação|      Analista De Bi| 5900.0|              0|              0|
|    2020-02-01|         13456|        1960-03-05|      2002-02-11|            null|   Ensino Médio Comp|    Assistente De Rh| 1400.0|              0|              0|
|    2020-07-01|         24457|        2000-07-26|      2020-05-10|            null|            Mestrado| Gerente De Produção|12100.0|              0|              0

In [0]:
%sql
-----[TESTE EM SQL] Incluindo a regra para criação da coluna idade
select mes_referencia,data_de_nascimento,
Case 
When data_de_nascimento <= '1800-01-01' then 0
else 
Cast(DATEDIFF(Day,data_de_nascimento, mes_referencia)/365 as numeric) 
End as Idade
from funcionarios 

mes_referencia,data_de_nascimento,Idade
2020-09-01,1982-02-12,39
2020-02-01,1960-03-05,60
2020-07-01,2000-07-26,20
2020-08-01,1998-05-17,22
2020-12-01,1972-01-11,49
2020-06-01,2000-07-26,20
2020-04-01,1995-04-01,25
2020-05-01,2000-07-26,20
2020-05-01,1991-05-28,29
2020-09-01,1960-03-05,61


In [0]:
# [APLICAÇÃO DE REGRA] Incluindo a regra para criação da coluna idade
df=df.withColumn("idade", expr("CASE WHEN data_de_nascimento <= '1800-01-01' THEN 0 ELSE Cast(DATEDIFF(Day,data_de_nascimento, mes_referencia)/365 as numeric) END"))
df.show(116)

+--------------+--------------+------------------+----------------+----------------+--------------------+--------------------+-------+---------------+---------------+-----+
|mes_referencia|Id_funcionario|data_de_nascimento|data_de_admissao|data_de_demissao|   grau_de_instrucao|               cargo|salario|admitido_no_mes|demitido_no_mes|idade|
+--------------+--------------+------------------+----------------+----------------+--------------------+--------------------+-------+---------------+---------------+-----+
|    2020-09-01|         46457|        1982-02-12|      1998-11-07|            null|       Pós Graduação|      Analista De Bi| 5900.0|              0|              0|   39|
|    2020-02-01|         13456|        1960-03-05|      2002-02-11|            null|   Ensino Médio Comp|    Assistente De Rh| 1400.0|              0|              0|   60|
|    2020-07-01|         24457|        2000-07-26|      2020-05-10|            null|            Mestrado| Gerente De Produção|12100.0| 

In [0]:
%sql
---- Analisando os dados unicos da coluna de grau de instrução

select distinct grau_de_instrucao

from funcionarios

grau_de_instrucao
Doutorado
Ensino Médio Comp
Ensino Fundamental Completo
Ensino Superior Completo
Ensino Médio Completo
4ª Série Completo
Pós Graduação
Mestrado
Graduação
Ensino Medio Incompleto


In [0]:
%sql
---- [TESTE EM SQL] Incluindo a regra para criação da coluna escolaridade categorizada
select grau_de_instrucao,
Case
when grau_de_instrucao like '%1%' or grau_de_instrucao like '%2%' or grau_de_instrucao like '%3%' or grau_de_instrucao like '%4%' or grau_de_instrucao like '%5%'
or grau_de_instrucao like '%6%' or grau_de_instrucao like '%7%' or grau_de_instrucao like '%8%' or grau_de_instrucao like '%9%' or grau_de_instrucao like '%Fund%inc%'
then 'Ensino Fundamental Incompleto'
when grau_de_instrucao like '%Fund%Com%' 
then 'Ensino Fundamental Completo'
when grau_de_instrucao like '%Med%Inco%'  or grau_de_instrucao like '%Méd%Inco%'
then 'Ensino Médio Incompleto'
when grau_de_instrucao like '%Med%Com%'  or  grau_de_instrucao like '%Méd%Com%'
then 'Ensino Médio Completo' 
when grau_de_instrucao like '%Sup%Com%'  or  grau_de_instrucao like 'Gra%'  
then 'Ensino Superior Completo'
when grau_de_instrucao like 'Pó%Gra%'    or  grau_de_instrucao like 'Po%Gra%'
then 'Pós Graduação'
when grau_de_instrucao like 'Dou%' 
then 'Doutorado'

when grau_de_instrucao like 'Mest%' 
then 'Mestrado'

end as escolaridade_categorizada

from funcionarios  


grau_de_instrucao,escolaridade_categorizada
Pós Graduação,Pós Graduação
Ensino Médio Comp,Ensino Médio Completo
Mestrado,Mestrado
Ensino Médio Completo,Ensino Médio Completo
Ensino Superior Completo,Ensino Superior Completo
Mestrado,Mestrado
4ª Série Completo,Ensino Fundamental Incompleto
Mestrado,Mestrado
Doutorado,Doutorado
Ensino Médio Comp,Ensino Médio Completo


In [0]:
# [APLICAÇÃO DE REGRA] Incluindo a regra para criação da coluna escolaridade categorizada
df=df.withColumn("escolaridade_categorizada", expr("CASE WHEN grau_de_instrucao like '%1%' or grau_de_instrucao like '%2%' or grau_de_instrucao like '%3%' or grau_de_instrucao like '%4%' or grau_de_instrucao like '%5%'or grau_de_instrucao like '%6%' or grau_de_instrucao like '%7%' or grau_de_instrucao like '%8%' or grau_de_instrucao like '%9%' or grau_de_instrucao like '%Fund%inc%' THEN 'Ensino Fundamental Incompleto'" + "WHEN grau_de_instrucao like '%Fund%Com%' THEN 'Ensino Fundamental Completo' "+ "WHEN grau_de_instrucao like '%Med%Inco%' or grau_de_instrucao like '%Méd%Inco%' THEN 'Ensino Médio Incompleto'"+ "WHEN grau_de_instrucao like '%Med%Com%'  or  grau_de_instrucao like '%Méd%Com%' THEN 'Ensino Médio Completo' "+"WHEN grau_de_instrucao like '%Sup%Com%'  or  grau_de_instrucao like 'Gra%' THEN 'Ensino Superior Completo' " + " WHEN grau_de_instrucao like 'Pó%Gra%' or  grau_de_instrucao like 'Po%Gra%' THEN 'Pós Graduação'" + "WHEN grau_de_instrucao like 'Dou%' THEN 'Doutorado'"+"WHEN grau_de_instrucao like 'Mest%' THEN 'Mestrado' END"))
display(df)

mes_referencia,Id_funcionario,data_de_nascimento,data_de_admissao,data_de_demissao,grau_de_instrucao,cargo,salario,admitido_no_mes,demitido_no_mes,idade,escolaridade_categorizada
2020-09-01,46457,1982-02-12,1998-11-07,null,Pós Graduação,Analista De Bi,5900.0,0,0,39,Pós Graduação
2020-02-01,13456,1960-03-05,2002-02-11,null,Ensino Médio Comp,Assistente De Rh,1400.0,0,0,60,Ensino Médio Completo
2020-07-01,24457,2000-07-26,2020-05-10,null,Mestrado,Gerente De Produção,12100.0,0,0,20,Mestrado
2020-08-01,59393,1998-05-17,2019-01-08,null,Ensino Médio Completo,Assistente Administrativo,1520.0,0,0,22,Ensino Médio Completo
2020-12-01,46578,1972-01-11,2009-11-26,null,Ensino Superior Completo,Analista De Sistemas Senior,8300.0,0,0,49,Ensino Superior Completo
2020-06-01,24457,2000-07-26,2020-05-10,null,Mestrado,Gerente De Produção,12100.0,0,0,20,Mestrado
2020-04-01,11223,1995-04-01,2018-06-01,null,4ª Série Completo,Operador De Colheita I,1500.0,0,0,25,Ensino Fundamental Incompleto
2020-05-01,24457,2000-07-26,2020-05-10,null,Mestrado,Gerente De Produção,11500.0,1,0,20,Mestrado
2020-05-01,57687,1991-05-28,2015-10-18,null,Doutorado,Cientista De Dados,7200.0,0,0,29,Doutorado
2020-09-01,13456,1960-03-05,2002-02-11,null,Ensino Médio Comp,Assistente De Rh,1520.0,0,0,61,Ensino Médio Completo


In [0]:
%sql
-----[TESTE EM SQL] Incluindo a regra para criação da coluna tempo de empresa
select mes_referencia,data_de_admissao,
Case 
When data_de_admissao <= '1800-01-01' then 0
else 
Cast(DATEDIFF(Month,data_de_admissao, mes_referencia) as integer) 
End as Tempo_de_empresa

from funcionarios 

mes_referencia,data_de_admissao,Tempo_de_empresa
2020-09-01,1998-11-07,261
2020-02-01,2002-02-11,215
2020-07-01,2020-05-10,1
2020-08-01,2019-01-08,18
2020-12-01,2009-11-26,132
2020-06-01,2020-05-10,0
2020-04-01,2018-06-01,22
2020-05-01,2020-05-10,0
2020-05-01,2015-10-18,54
2020-09-01,2002-02-11,222


In [0]:
# [APLICAÇÃO DE REGRA] Incluindo a regra para criação da coluna tempo de empresa
df=df.withColumn("tempo_de_empresa", expr("CASE WHEN data_de_admissao <= '1800-01-01' THEN 0 ELSE Cast(DATEDIFF(Month,data_de_admissao, mes_referencia) as integer)   END"))
display(df)

mes_referencia,Id_funcionario,data_de_nascimento,data_de_admissao,data_de_demissao,grau_de_instrucao,cargo,salario,admitido_no_mes,demitido_no_mes,idade,escolaridade_categorizada,tempo_de_empresa
2020-09-01,46457,1982-02-12,1998-11-07,null,Pós Graduação,Analista De Bi,5900.0,0,0,39,Pós Graduação,261
2020-02-01,13456,1960-03-05,2002-02-11,null,Ensino Médio Comp,Assistente De Rh,1400.0,0,0,60,Ensino Médio Completo,215
2020-07-01,24457,2000-07-26,2020-05-10,null,Mestrado,Gerente De Produção,12100.0,0,0,20,Mestrado,1
2020-08-01,59393,1998-05-17,2019-01-08,null,Ensino Médio Completo,Assistente Administrativo,1520.0,0,0,22,Ensino Médio Completo,18
2020-12-01,46578,1972-01-11,2009-11-26,null,Ensino Superior Completo,Analista De Sistemas Senior,8300.0,0,0,49,Ensino Superior Completo,132
2020-06-01,24457,2000-07-26,2020-05-10,null,Mestrado,Gerente De Produção,12100.0,0,0,20,Mestrado,0
2020-04-01,11223,1995-04-01,2018-06-01,null,4ª Série Completo,Operador De Colheita I,1500.0,0,0,25,Ensino Fundamental Incompleto,22
2020-05-01,24457,2000-07-26,2020-05-10,null,Mestrado,Gerente De Produção,11500.0,1,0,20,Mestrado,0
2020-05-01,57687,1991-05-28,2015-10-18,null,Doutorado,Cientista De Dados,7200.0,0,0,29,Doutorado,54
2020-09-01,13456,1960-03-05,2002-02-11,null,Ensino Médio Comp,Assistente De Rh,1520.0,0,0,61,Ensino Médio Completo,222


In [0]:
%sql 
--- [TESTE EM SQL] Incluindo a regra para criação da coluna mudou de salario 
select mes_referencia,Id_funcionario,salario ,

LAG(0,1,1)  OVER (PARTITION BY id_funcionario,salario ORDER BY mes_referencia,id_funcionario) as  mudou_de_salario
       
from funcionarios 
order by 1,2

mes_referencia,Id_funcionario,salario,mudou_de_salario
2020-01-01,11223,1500.0,1
2020-01-01,12345,1800.0,1
2020-01-01,13254,1625.0,1
2020-01-01,13456,1400.0,1
2020-01-01,24457,11500.0,1
2020-01-01,46457,5900.0,1
2020-01-01,46578,7500.0,1
2020-01-01,57687,7200.0,1
2020-01-01,59393,1400.0,1
2020-01-01,95847,11800.0,1


In [0]:
# [APLICAÇÃO DE REGRA] Incluindo a regra para criação da coluna mudou de salario 
df=df.withColumn("mudou_de_salario", expr("LAG(0,1,1)  OVER (PARTITION BY id_funcionario,salario ORDER BY mes_referencia,id_funcionario) "))

display(df)



mes_referencia,Id_funcionario,data_de_nascimento,data_de_admissao,data_de_demissao,grau_de_instrucao,cargo,salario,admitido_no_mes,demitido_no_mes,idade,escolaridade_categorizada,tempo_de_empresa,mudou_de_salario
2020-01-01,11223,1995-04-01,2018-06-01,null,4ª Série Completo,Operador De Colheita I,1500.0,0,0,25,Ensino Fundamental Incompleto,19,1
2020-02-01,11223,1995-04-01,2018-06-01,null,4ª Série Completo,Operador De Colheita I,1500.0,0,0,25,Ensino Fundamental Incompleto,20,0
2020-03-01,11223,1995-04-01,2018-06-01,null,4ª Série Completo,Operador De Colheita I,1500.0,0,0,25,Ensino Fundamental Incompleto,21,0
2020-04-01,11223,1995-04-01,2018-06-01,null,4ª Série Completo,Operador De Colheita I,1500.0,0,0,25,Ensino Fundamental Incompleto,22,0
2020-05-01,11223,1995-04-01,2018-06-01,null,4ª Série Completo,Operador De Colheita I,1500.0,0,0,25,Ensino Fundamental Incompleto,23,0
2020-06-01,11223,1995-04-01,2018-06-01,null,4ª Série Completo,Operador De Colheita I,1500.0,0,0,25,Ensino Fundamental Incompleto,24,0
2020-07-01,11223,1995-04-01,2018-06-01,null,4ª Série Completo,Operador De Colheita I,1500.0,0,0,25,Ensino Fundamental Incompleto,25,0
2020-08-01,11223,1995-04-01,2018-06-01,null,4ª Série Completo,Operador De Colheita I,1500.0,0,0,25,Ensino Fundamental Incompleto,26,0
2020-09-01,11223,1995-04-01,2018-06-01,null,4ª Série Completo,Operador De Colheita Ii,1750.0,0,0,25,Ensino Fundamental Incompleto,27,1
2020-10-01,11223,1995-04-01,2018-06-01,null,4ª Série Completo,Operador De Colheita Ii,1750.0,0,0,26,Ensino Fundamental Incompleto,28,0


In [0]:
%sql 
--- [TESTE EM SQL]  Incluindo a regra para criação da coluna mudou de cargo
select mes_referencia,Id_funcionario,cargo ,

LAG(0,1,1)  OVER (PARTITION BY id_funcionario,cargo ORDER BY mes_referencia,id_funcionario) as  mudou_de_cargo
       
from funcionarios 
order by 1,2

mes_referencia,Id_funcionario,cargo,mudou_de_cargo
2020-01-01,11223,Operador De Colheita I,1
2020-01-01,12345,Operador De Colheita Ii,1
2020-01-01,13254,Operador De Colheita I,1
2020-01-01,13456,Assistente De Rh,1
2020-01-01,24457,Gerente De Produção,1
2020-01-01,46457,Analista De Bi,1
2020-01-01,46578,Analista De Sistemas Pleno,1
2020-01-01,57687,Cientista De Dados,1
2020-01-01,59393,Assistente Administrativo,1
2020-01-01,95847,Líder De Tecnologia,1


In [0]:
# [APLICAÇÃO DE REGRA]  Incluindo a regra para criação da coluna mudou de cargo
df=df.withColumn("mudou_de_cargo", expr("LAG(0,1,1)  OVER (PARTITION BY id_funcionario,cargo ORDER BY mes_referencia,id_funcionario)"))

display(df)


mes_referencia,Id_funcionario,data_de_nascimento,data_de_admissao,data_de_demissao,grau_de_instrucao,cargo,salario,admitido_no_mes,demitido_no_mes,idade,escolaridade_categorizada,tempo_de_empresa,mudou_de_salario,mudou_de_cargo
2020-01-01,11223,1995-04-01,2018-06-01,null,4ª Série Completo,Operador De Colheita I,1500.0,0,0,25,Ensino Fundamental Incompleto,19,1,1
2020-02-01,11223,1995-04-01,2018-06-01,null,4ª Série Completo,Operador De Colheita I,1500.0,0,0,25,Ensino Fundamental Incompleto,20,0,0
2020-03-01,11223,1995-04-01,2018-06-01,null,4ª Série Completo,Operador De Colheita I,1500.0,0,0,25,Ensino Fundamental Incompleto,21,0,0
2020-04-01,11223,1995-04-01,2018-06-01,null,4ª Série Completo,Operador De Colheita I,1500.0,0,0,25,Ensino Fundamental Incompleto,22,0,0
2020-05-01,11223,1995-04-01,2018-06-01,null,4ª Série Completo,Operador De Colheita I,1500.0,0,0,25,Ensino Fundamental Incompleto,23,0,0
2020-06-01,11223,1995-04-01,2018-06-01,null,4ª Série Completo,Operador De Colheita I,1500.0,0,0,25,Ensino Fundamental Incompleto,24,0,0
2020-07-01,11223,1995-04-01,2018-06-01,null,4ª Série Completo,Operador De Colheita I,1500.0,0,0,25,Ensino Fundamental Incompleto,25,0,0
2020-08-01,11223,1995-04-01,2018-06-01,null,4ª Série Completo,Operador De Colheita I,1500.0,0,0,25,Ensino Fundamental Incompleto,26,0,0
2020-09-01,11223,1995-04-01,2018-06-01,null,4ª Série Completo,Operador De Colheita Ii,1750.0,0,0,25,Ensino Fundamental Incompleto,27,1,1
2020-10-01,11223,1995-04-01,2018-06-01,null,4ª Série Completo,Operador De Colheita Ii,1750.0,0,0,26,Ensino Fundamental Incompleto,28,0,0


In [0]:
# Ordenando o dataframe 
df=df.orderBy(df.mes_referencia.asc())

In [0]:
display(df)

mes_referencia,Id_funcionario,data_de_nascimento,data_de_admissao,data_de_demissao,grau_de_instrucao,cargo,salario,admitido_no_mes,demitido_no_mes,idade,escolaridade_categorizada,tempo_de_empresa,mudou_de_salario,mudou_de_cargo
2020-01-01,57687,1991-05-28,2015-10-18,null,Doutorado,Cientista De Dados,7200.0,0,0,29,Doutorado,50,1,1
2020-01-01,11223,1995-04-01,2018-06-01,null,4ª Série Completo,Operador De Colheita I,1500.0,0,0,25,Ensino Fundamental Incompleto,19,1,1
2020-01-01,13456,1960-03-05,2002-02-11,null,Ensino Médio Comp,Assistente De Rh,1400.0,0,0,60,Ensino Médio Completo,214,1,1
2020-01-01,59393,1998-05-17,2019-01-08,null,Ensino Médio Completo,Assistente Administrativo,1400.0,0,0,22,Ensino Médio Completo,11,1,1
2020-01-01,95847,1993-04-23,2012-04-25,null,Graduação,Líder De Tecnologia,11800.0,0,0,27,Ensino Superior Completo,92,1,1
2020-01-01,12345,1980-11-02,1800-01-01,null,Ensino Fundamental Completo,Operador De Colheita Ii,1800.0,0,0,39,Ensino Fundamental Completo,0,1,1
2020-01-01,24457,2000-07-26,2020-05-10,null,Mestrado,Gerente De Produção,11500.0,0,0,19,Mestrado,-4,1,1
2020-01-01,46457,1982-02-12,1998-11-07,null,Pós Graduação,Analista De Bi,5900.0,0,0,38,Pós Graduação,253,1,1
2020-01-01,13254,1977-05-22,2005-09-03,null,Ensino Medio Incompleto,Operador De Colheita I,1625.0,0,0,43,Ensino Médio Incompleto,171,1,1
2020-01-01,46578,1972-01-11,2009-11-26,null,Ensino Superior Completo,Analista De Sistemas Pleno,7500.0,0,0,48,Ensino Superior Completo,121,1,1


In [0]:
# Verificando o schema da tabela completa após as criações das colunas adicionais
df.printSchema()

root
 |-- mes_referencia: date (nullable = true)
 |-- Id_funcionario: integer (nullable = true)
 |-- data_de_nascimento: date (nullable = true)
 |-- data_de_admissao: date (nullable = true)
 |-- data_de_demissao: date (nullable = true)
 |-- grau_de_instrucao: string (nullable = false)
 |-- cargo: string (nullable = false)
 |-- salario: float (nullable = false)
 |-- admitido_no_mes: integer (nullable = false)
 |-- demitido_no_mes: integer (nullable = false)
 |-- idade: decimal(10,0) (nullable = true)
 |-- escolaridade_categorizada: string (nullable = true)
 |-- tempo_de_empresa: integer (nullable = true)
 |-- mudou_de_salario: integer (nullable = false)
 |-- mudou_de_cargo: integer (nullable = false)

